<a href="https://colab.research.google.com/github/armandossrecife/mysummary/blob/main/Resumo_processos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resumo de Fragmentos de Processos usando IA

Protótipo para testar resumos de fragmentos de processos

Dado um arquivo PDF relativo a um processo público faz o resumo do texto de um *fragmento selecionado neste processo*

**Objetivo**: Fazer o uso de uma [LLM](https://pt.wikipedia.org/wiki/Modelos_de_linguagem_de_grande_escala) ([IA Generativa](https://pt.wikipedia.org/wiki/Intelig%C3%AAncia_artificial_generativa): [Google Gemini](https://gemini.google.com)) para resumir fragmentos de processos públicos.

**Observação**: o protótipo é apenas um teste de "prova de conceito", não foi levado em consideração o método científico para validar o design do protótipo. Além disso, os resultados não foram validados por um especialista (Advogado).

Etapas do teste do protótipo:

1. Acesso aos processos públicos do STJ via https://processo.stj.jus.br/processo/pesquisa

2. Processos acessados: (escolhidos aleatoriamente)
- P1 (HABEAS CORPUS No 894602 - SP (2024/0066576-2))
- P2 (HABEAS CORPUS No 894601 - SE (2024/0066524-4))
- P3 (AGRAVO EM RECURSO ESPECIAL No 2576723 - CE (2024/0064554-2))

3. Resultados:
- Disponíveis no tópico "Execução do Protótipo"

## Programação do Protótipo (Prova de Conceito)

### Código-fonte da Aplicação Base

In [1]:
!pip install -q -U google-generativeai
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.0 MB/s eta 0:00:00


In [34]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [38]:
# Definição de um modelo LLM Genérico
model = genai.GenerativeModel('gemini-pro')

In [39]:
# Definição das funções de apoio
import PyPDF2
import os
from datetime import datetime

def extract_pdf_text(pdf_path):
    try:
        with open(pdf_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)

            pdf_text = {}
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                pdf_text[page_num + 1] = text
            return pdf_text
    except FileNotFoundError:
        print(f"Error: PDF file not found at {pdf_path}")
    except PermissionError:
        print(f"Error: Permission denied to access PDF file at {pdf_path}")
    except Exception as e:
        print(f"Error: An unexpected error occurred: {e}")

def get_summary_from_response(response_resumo_texto):
  try:
    if len(response_resumo_texto.candidates) > 0:
      summary_content = response_resumo_texto.candidates[0].content
      if len(summary_content.parts) > 0:
        summary_text = summary_content.parts[0].text
        return summary_text  # Return the extracted text
      else:
        print("No content parts found in the first candidate.")
    else:
      print("Nenhuma resposta disponível")
  except (AttributeError, IndexError):
    print("Error accessing response data. Please check the response structure.")

def request_to_gemini(prompt_text):
  print("Aguarde...")
  t1 = datetime.now()
  response_resumo_texto = model.generate_content(prompt_text)
  t2 = datetime.now()
  print(response_resumo_texto)
  tempo_resposta = t2 - t1
  print(f"Tempo de resposta: {tempo_resposta}")
  return response_resumo_texto

def count_words(text):
    words = text.split()
    return len(words)

def processa_resumo(nome_arquivo, pergunte_ao_gemini):
  texto_documento = ""
  meu_conteudo =  extract_pdf_text(pdf_path=nome_arquivo)
  print(f"Quantidade de páginas: {len(meu_conteudo.keys())}")
  for k,v in meu_conteudo.items():
    texto_documento = texto_documento + v
  quantidade_palavras = count_words(texto_documento)
  print(f"Quantidade de palavras: {quantidade_palavras}")
  prompt_text = f"{pergunte_ao_gemini}{texto_documento}"
  response_resumo_texto = request_to_gemini(prompt_text)
  return response_resumo_texto

## Execução do Protótipo

Os seguintes processos foram usados:
- p1 (HABEAS CORPUS No 894602 - SP (2024/0066576-2))
- p2 (HABEAS CORPUS No 894601 - SE (2024/0066524-4))
- p3 (AGRAVO EM RECURSO ESPECIAL No 2576723 - CE (2024/0064554-2))

### Leitura dos Fragmentos dos Processos

Entre o com os documentos (PDFs) dos fragmentos dos processos

In [37]:
!wget https://raw.githubusercontent.com/armandossrecife/mysummary/refs/heads/main/pdfs/stj_dje_20240306_0_40501136.pdf
!wget https://raw.githubusercontent.com/armandossrecife/mysummary/refs/heads/main/pdfs/stj_dje_20240621_0_42069415.pdf
!wget https://raw.githubusercontent.com/armandossrecife/mysummary/refs/heads/main/pdfs/stj_dje_20240612_0_41924605.pdf

--2024-10-31 12:18:47--  https://raw.githubusercontent.com/armandossrecife/mysummary/refs/heads/main/pdfs/stj_dje_20240306_0_40501136.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160388 (157K) [application/octet-stream]
Saving to: ‘stj_dje_20240306_0_40501136.pdf’

stj_dje_20240306_0_ 100%[===================>] 156.63K  --.-KB/s    in 0.02s   

2024-10-31 12:18:47 (6.66 MB/s) - ‘stj_dje_20240306_0_40501136.pdf’ saved [160388/160388]

--2024-10-31 12:18:48--  https://raw.githubusercontent.com/armandossrecife/mysummary/refs/heads/main/pdfs/stj_dje_20240621_0_42069415.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.

### Resumo do 1o. Processo

In [42]:
resposta_resumo_p1 = processa_resumo(nome_arquivo="stj_dje_20240306_0_40501136.pdf", pergunte_ao_gemini="Você pode fazer um resumo do seguinte texto: ")
meu_resumo_p1 = get_summary_from_response(resposta_resumo_p1)
print(meu_resumo_p1)

Quantidade de páginas: 3
Quantidade de palavras: 875
Aguarde...
response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "**Resumo do Habeas Corpus:**\n\nO Superior Tribunal de Justi\u00e7a (STJ) indeferiu liminarmente um pedido de habeas corpus impetrado em favor de um paciente que cumpre pena em regime semiaberto por delitos de roubo e estupro.\n\nA defesa do paciente alegou que ele sofre de graves problemas de sa\u00fade e pediu a concess\u00e3o de pris\u00e3o domiciliar humanit\u00e1ria. No entanto, as inst\u00e2ncias anteriores n\u00e3o reconheceram flagrante ilegalidade no indeferimento do pedido.\n\nO STJ aplicou a S\u00famula 691 do Supremo Tribunal Federal, que impede a an\u00e1lise de habeas corpus contra decis\u00e3o de relator que indeferir liminar em habeas corpus requerido a tribunal superior.\n\nO ST

In [46]:
quantidade_palavras1 = count_words(meu_resumo_p1)
print(f"Quantidade de palavras do resumo (Processo1): {quantidade_palavras1}")

Quantidade de palavras do resumo (Processo1): 145


### Resumo do 2o. Processo

In [41]:
resposta_resumo_p2 = processa_resumo(nome_arquivo="stj_dje_20240621_0_42069415.pdf", pergunte_ao_gemini="Você pode fazer um resumo do seguinte texto: ")
meu_resumo_p2 = get_summary_from_response(resposta_resumo_p2)
print(meu_resumo_p2)

Quantidade de páginas: 2
Quantidade de palavras: 596
Aguarde...
response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "**Resumo do Habeas Corpus:**\n\nO habeas corpus n\u00ba 894601-SE foi impetrado em benef\u00edcio de Kaique Batista Santos, preso preventivamente por suposto tr\u00e1fico de drogas. O Tribunal de Justi\u00e7a do Estado de Sergipe foi apontado como autoridade coatora.\n\nO paciente alega que sua pris\u00e3o foi decretada pelo juiz de plant\u00e3o de forma arbitr\u00e1ria, sem representa\u00e7\u00e3o da autoridade policial ou do Minist\u00e9rio P\u00fablico. Al\u00e9m disso, a fundamenta\u00e7\u00e3o se baseou apenas na gravidade abstrata do crime e n\u00e3o em elementos concretos do caso espec\u00edfico.\n\nO relator, Ministro Sebasti\u00e3o Reis J\u00fanior, concedeu a ordem de habeas corpus, re

In [45]:
quantidade_palavras2 = count_words(meu_resumo_p2)
print(f"Quantidade de palavras do resumo (Processo2): {quantidade_palavras2}")

Quantidade de palavras do resumo (Processo2): 159


### Resumo do 3o. Processo

In [40]:
resposta_resumo_p3 = processa_resumo(nome_arquivo="stj_dje_20240612_0_41924605.pdf", pergunte_ao_gemini="Você pode fazer um resumo do seguinte texto: ")
meu_resumo_p3 = get_summary_from_response(resposta_resumo_p3)
print(meu_resumo_p3)

Quantidade de páginas: 5
Quantidade de palavras: 2255
Aguarde...
response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "**Resumo do Agravo em Recurso Especial**\n\n**Relator:** Ministro Herman Benjamin\n\n**Agravante:** Central E\u00f3lica S\u00e3o Raimundo S.A.\n\n**Agravados:** Raimundo Pedro da Silva e Maria Auxiliadora da Silva\n\n**Interessada:** Alian\u00e7a Gera\u00e7\u00e3o de Energia S.A.\n\n**Decis\u00e3o:** N\u00e3o conhecimento do agravo\n\n**S\u00edntese:**\n\nA agravante interp\u00f4s recurso especial contra decis\u00e3o que desconstituiu senten\u00e7a de extin\u00e7\u00e3o do processo por decad\u00eancia e prescri\u00e7\u00e3o em uma a\u00e7\u00e3o declarat\u00f3ria de nulidade contratual envolvendo uma servid\u00e3o administrativa.\n\nA agravante alegou viola\u00e7\u00e3o do art. 178, II, do C\u0

In [47]:
quantidade_palavras3 = count_words(meu_resumo_p3)
print(f"Quantidade de palavras do resumo (Processo3): {quantidade_palavras3}")

Quantidade de palavras do resumo (Processo3): 186
